In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
PATH = '/content/drive/MyDrive/Colab Notebooks/ПМ-2022/АД/data/wine+quality.zip'

In [5]:
import zipfile

In [6]:
zf = zipfile.ZipFile(PATH)

In [16]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.8 MB/s eta 0:00:00


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import balanced_accuracy_score

In [28]:
from sklearn.model_selection import GridSearchCV

#Загрузка данных

In [7]:
df_red = pd.read_csv(zf.open('winequality-red.csv'), sep=";")
df_red["target"] = 1
df_white = pd.read_csv(zf.open('winequality-white.csv'), sep=";")
df_white["target"] = 0

In [8]:
df_red.shape, df_white.shape

((1599, 13), (4898, 13))

In [9]:
df = pd.concat([df_red, df_white], ignore_index = True)

In [11]:
df.shape

(6497, 13)

In [12]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,target
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


#Описательный анализ

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
 12  target                6497 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 660.0 KB


In [14]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,target
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.215307,0.339666,0.318633,5.443235,0.056034,30.525319,115.744574,0.994697,3.218501,0.531268,10.491801,5.818378,0.246114
std,1.296434,0.164636,0.145318,4.757804,0.035034,17.749400,56.521855,0.002999,0.160787,0.148806,1.192712,0.873255,0.430779
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000,0.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000,0.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000,0.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000,0.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000,1.000000


In [15]:
df['target'].value_counts(normalize=True)

,proportion
target,
0,0.753886
1,0.246114


#Предобработка

In [19]:
df.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality', 'target'],
      dtype='object')

In [20]:
X = df.drop('target', axis=1)
y = df['target']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

#Деревья

In [25]:
dtc1 = DecisionTreeClassifier(max_depth=3)
dtc1.fit(X_train, y_train)
dtc1_y_pred = dtc1.predict(X_test)
dtc1_y_pred_train = dtc1.predict(X_train)
print(f"Train: {balanced_accuracy_score(y_train, dtc1_y_pred_train)}; Test: {balanced_accuracy_score(y_test, dtc1_y_pred)}")

Train: 0.9715163885696168; Test: 0.964860172569391


In [26]:
dtc2 = DecisionTreeClassifier(max_depth=5)
dtc2.fit(X_train, y_train)
dtc2_y_pred = dtc2.predict(X_test)
dtc2_y_pred_train = dtc2.predict(X_train)
print(f"Train: {balanced_accuracy_score(y_train, dtc2_y_pred_train)}; Test: {balanced_accuracy_score(y_test, dtc2_y_pred)}")

Train: 0.9848980751192317; Test: 0.9746346107283573


In [27]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
dtc_y_pred = dtc.predict(X_test)
dtc_y_pred_train = dtc.predict(X_train)
print(f"Train: {balanced_accuracy_score(y_train, dtc_y_pred_train)}; Test: {balanced_accuracy_score(y_test, dtc_y_pred)}")

Train: 0.9990884229717412; Test: 0.9819121304835905


In [33]:
parameters = {'max_depth': range(3,20,2),
              'min_samples_split': range(2,10),
              'min_samples_leaf': range(1,10)}
dtc = DecisionTreeClassifier()
clf = GridSearchCV(dtc, parameters, cv=5, scoring="balanced_accuracy")
clf.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(3, 20, 2),
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10)},
             scoring='balanced_accuracy')

In [34]:
clf.best_params_, clf.best_score_

({'max_depth': 15, 'min_samples_leaf': 3, 'min_samples_split': 5},
 0.9787602649484721)

#Лес

In [42]:
rfc1 = RandomForestClassifier(n_estimators = 100, max_depth=1)
rfc1.fit(X_train, y_train)
rfc1_y_pred = rfc1.predict(X_test)
rfc1_y_pred_train = rfc1.predict(X_train)
print(f"Train: {balanced_accuracy_score(y_train, rfc1_y_pred_train)}; Test: {balanced_accuracy_score(y_test, rfc1_y_pred)}")

Train: 0.8629945965941368; Test: 0.8469189540181814


#Бустинги

In [43]:
model = XGBClassifier(n_estimators=100, max_depth=3, learning_rate=0.1)

model.fit(X_train, y_train)
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

print(f'Точность BAC на обучающей выборке: {balanced_accuracy_score(pred_train, y_train)}')
print(f'Точность BAC на валидационной выборке: {balanced_accuracy_score(pred_test, y_test)}')

Точность BAC на обучающей выборке: 0.9991319444444444
Точность BAC на валидационной выборке: 0.9952328393308781


In [46]:
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=3)
model.fit(X_train, y_train)
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

print(f'Точность BAC на обучающей выборке: {balanced_accuracy_score(pred_train, y_train)}')
print(f'Точность BAC на валидационной выборке: {balanced_accuracy_score(pred_test, y_test)}')

Выходные данные были обрезаны до нескольких последних строк (5000).
5002:	learn: 0.0021583	total: 12s	remaining: 12s
5003:	learn: 0.0021583	total: 12s	remaining: 12s
5004:	learn: 0.0021583	total: 12s	remaining: 12s
5005:	learn: 0.0021583	total: 12s	remaining: 12s
5006:	learn: 0.0021582	total: 12s	remaining: 12s
5007:	learn: 0.0021582	total: 12s	remaining: 11.9s
5008:	learn: 0.0021582	total: 12s	remaining: 11.9s
5009:	learn: 0.0021581	total: 12s	remaining: 11.9s
5010:	learn: 0.0021582	total: 12s	remaining: 11.9s
5011:	learn: 0.0021582	total: 12s	remaining: 11.9s
5012:	learn: 0.0021581	total: 12s	remaining: 11.9s
5013:	learn: 0.0021581	total: 12s	remaining: 11.9s
5014:	learn: 0.0021581	total: 12s	remaining: 11.9s
5015:	learn: 0.0021581	total: 12s	remaining: 11.9s
5016:	learn: 0.0021581	total: 12s	remaining: 11.9s
5017:	learn: 0.0021581	total: 12s	remaining: 11.9s
5018:	learn: 0.0021580	total: 12s	remaining: 11.9s
5019:	learn: 0.0021580	total: 12s	remaining: 11.9s
5020:	learn: 0.0021580	t